In [ ]:
# install the library

from zoraspeech import Listener
import numpy as np

In [ ]:
# set up autoreload

%load_ext autoreload
%autoreload 2

In [ ]:
# create a listener

listener = Listener(model_architecture=None, model_weights=None)

In [ ]:

# create a sample audio buffer

audio_buffer = np.random.rand(1000)

# listen to the audio buffer 

listener.listen(audio_buffer)